# Amazon SageMaker Batch Transform

## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [15]:
!python -m pip install --upgrade pip --quiet
!pip install -U awscli --quiet
!pip install sagemaker --upgrade

In [21]:
import os
import boto3
import sagemaker
from time import gmtime, strftime
from datetime import datetime

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_role = sagemaker.get_execution_role()
region = boto_session.region_name
s3_bucket = sm_session.default_bucket()
bucket_prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"
resource_name = "BatchInferenceDemo-{}-{}"

print(f"Will use bucket '{s3_bucket}' for storing all resources related to this notebook")
print(f"Using Role: {sm_role}")

Will use bucket 'sagemaker-us-west-2-757967535041' for storing all resources related to this notebook
Using Role: arn:aws:iam::757967535041:role/service-role/AmazonSageMaker-ExecutionRole-20210212T154271


---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [34]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file("sagemaker-sample-files", "datasets/tabular/breast_cancer/wdbc.csv", filename)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
307,89346,B,9.00,14.40,56.36,246.3,0.07005,0.03116,0.003681,0.003472,...,9.699,20.07,60.90,285.5,0.09861,0.05232,0.01472,0.01389,0.2991,0.07804
343,898431,M,19.68,21.68,129.90,1194.0,0.09797,0.13390,0.186300,0.110300,...,22.750,34.66,157.60,1540.0,0.12180,0.34580,0.47340,0.22550,0.4045,0.07918
522,91789,B,11.26,19.83,71.30,388.1,0.08511,0.04413,0.005067,0.005664,...,11.930,26.43,76.38,435.9,0.11080,0.07723,0.02533,0.02832,0.2557,0.07613
79,8610908,B,12.86,18.00,83.19,506.3,0.09934,0.09546,0.038890,0.023150,...,14.240,24.82,91.88,622.1,0.12890,0.21410,0.17310,0.07926,0.2779,0.07918
225,88143502,B,14.34,13.47,92.51,641.2,0.09906,0.07624,0.057240,0.046030,...,16.770,16.90,110.40,873.2,0.12970,0.15250,0.16320,0.10870,0.3062,0.06072
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.241400,0.105200,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.68690,0.25750,0.6638,0.17300
277,8911670,M,18.81,19.98,120.90,1102.0,0.08923,0.05884,0.080200,0.058430,...,19.960,24.30,129.00,1236.0,0.12430,0.11600,0.22100,0.12940,0.2567,0.05737
454,911202,B,12.62,17.15,80.62,492.9,0.08583,0.05430,0.029660,0.022720,...,14.340,22.15,91.62,633.5,0.12250,0.15170,0.18870,0.09851,0.3270,0.07330


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [35]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
491,91376702,0,17.850,13.23,114.60,992.1,0.07838,0.06217,0.04445,0.04178,...,19.820,18.42,127.10,1210.0,0.09862,0.09976,0.1048,0.08341,0.1783,0.05871
389,90312,1,19.550,23.21,128.90,1174.0,0.10100,0.13180,0.18560,0.10210,...,20.820,30.44,142.00,1313.0,0.12510,0.24140,0.3829,0.18250,0.2576,0.07602
174,871642,0,10.660,15.15,67.49,349.6,0.08792,0.04302,0.00000,0.00000,...,11.540,19.20,73.20,408.3,0.10760,0.06791,0.0000,0.00000,0.2710,0.06164
205,879523,1,15.120,16.68,98.78,716.6,0.08876,0.09588,0.07550,0.04079,...,17.770,20.24,117.70,989.5,0.14910,0.33310,0.3327,0.12520,0.3415,0.09740
192,875099,0,9.720,18.22,60.73,288.1,0.06950,0.02344,0.00000,0.00000,...,9.968,20.83,62.25,303.8,0.07117,0.02729,0.0000,0.00000,0.1909,0.06559
140,868999,0,9.738,11.97,61.24,288.5,0.09250,0.04102,0.00000,0.00000,...,10.620,14.10,66.53,342.9,0.12340,0.07204,0.0000,0.00000,0.3105,0.08151
255,886452,1,13.960,17.05,91.43,602.4,0.10960,0.12790,0.09789,0.05246,...,16.390,22.07,108.10,826.0,0.15120,0.32620,0.3209,0.13740,0.3068,0.07957
72,859717,1,17.200,24.52,114.20,929.4,0.10710,0.18300,0.16920,0.07944,...,23.320,33.82,151.60,1681.0,0.15850,0.73940,0.6566,0.18990,0.3313,0.13390


Let's split the data and set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

Let's upload those data sets in S3

In [36]:
# train_file = "train_data.csv"
# data_train.to_csv(train_file, index=False, header=False)
# sess.upload_data(train_file, key_prefix="{}/train".format(bucket_prefix))

# validation_file = "validation_data.csv"
# data_val.to_csv(validation_file, index=False, header=False)
# sess.upload_data(validation_file, key_prefix="{}/validation".format(bucket_prefix))

batch_list = rand_split >= 0.9
data_batch = data[batch_list].drop(["diagnosis"], axis=1)

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(bucket_prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(bucket_prefix))

's3://sagemaker-us-west-2-757967535041/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

## Create a SageMaker Model

Specify the location of the pre-trained model stored in Amazon S3. This example uses a pre-trained XGBoost model name demo-xgboost-model.tar.gz. The full Amazon S3 URI is stored in a string variable model_url.

In [37]:
model_s3_key = f"{bucket_prefix}/model.tar.gz"
model_url = f"s3://{s3_bucket}/{model_s3_key}"
print(f"Uploading Model to {model_url}")

with open("model/model.tar.gz", "rb") as model_file:
    boto_session.resource("s3").Bucket(s3_bucket).Object(model_s3_key).upload_fileobj(model_file)

Uploading Model to s3://sagemaker-us-west-2-757967535041/DEMO-breast-cancer-prediction-xgboost-highlevel/model.tar.gz


Specify a primary container. For the primary container, you specify the Docker image that contains inference code, artifacts (from prior training), and a custom environment map that the inference code uses when you deploy the model for predictions. In this example, we specify an XGBoost built-in algorithm container image.

In [38]:
from sagemaker import image_uris

# Specify an AWS container image and region as desired
container = image_uris.retrieve(region=region, framework="xgboost", version="0.90-1")

Create a SageMaker Model by specifying the name, the role (the ARN of the IAM role that Amazon SageMaker can assume to access model artifacts/ docker images for deployment), and the image_uri of the XGBoost built-in algorithm container image.

In [39]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model_name = resource_name.format("Model", datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

model_predictor = Model(
    name=model_name,
    image_uri=container,
    model_data=model_url,
    role=sm_role,
    predictor_cls=Predictor,
)
model_name

'BatchInferenceDemo-Model-2022-07-28-18-58-51'

In [40]:
# Deploy the model
instance_count=1,
instance_type="ml.m5.4xlarge"

predictor = model_predictor.deploy(
        instance_type='ml.m5.4xlarge',
        initial_instance_count=1)


----!

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [41]:
# %%time

from sagemaker.transformer import Transformer

sm_transformer = Transformer(model_name=model_name,
                          instance_count=1,
                          instance_type='ml.m4.xlarge')

#Start the Batch Transform job

input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column

sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")


....................................
[2022-07-28 19:06:44 +0000] [14] [INFO] Starting gunicorn 19.10.0
[2022-07-28 19:06:44 +0000] [14] [INFO] Listening at: unix:/tmp/gunicorn.sock (14)
[2022-07-28 19:06:44 +0000] [14] [INFO] Using worker: gevent
[2022-07-28 19:06:44 +0000] [21] [INFO] Booting worker with pid: 21
[2022-07-28 19:06:44 +0000] [22] [INFO] Booting worker with pid: 22
[2022-07-28 19:06:44 +0000] [23] [INFO] Booting worker with pid: 23
[2022-07-28 19:06:44 +0000] [24] [INFO] Booting worker with pid: 24
[2022-07-28:19:06:51:INFO] No GPUs detected (normal if no gpus installed)
[2022-07-28:19:06:51:INFO] No GPUs detected (normal if no gpus installed)
169.254.255.130 - - [28/Jul/2022:19:06:51 +0000] "GET /ping HTTP/1.1" 200 0 "-" "Go-http-client/1.1"
[2022-07-28:19:06:51:INFO] No GPUs detected (normal if no gpus installed)
169.254.255.130 - - [28/Jul/2022:19:06:51 +0000] "GET /execution-parameters HTTP/1.1" 200 84 "-" "Go-http-client/1.1"
[2022-07-28:19:06:51:INFO] No GPUs detec

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [42]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [43]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.952333
1,0.886067
2,0.990161
3,0.946707
4,0.912574
5,0.987773
6,0.989698
7,0.006414
